In [1]:
import os
from importlib.resources import files
import time
import numpy as np
import torch
%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt
# Enable LaTeX rendering and set Times as the font family
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'Times'
from dtaidistance import dtw_ndim
from learn_diffeomorphism import Dynamics

from learn_embedding.approximators import *
from learn_embedding.covariances import *
from learn_embedding.kernels import SquaredExp
from learn_embedding.embedding import Embedding
from learn_embedding.dynamics import FirstGeometry, SecondGeometry
from learn_embedding.utils import *
from learn_embedding.loss import *

In [2]:
# name
demo_name = 'demo1'
# problem dimension
dim = 2
# Total number of trajectories
num_traj = 7
# Number of trajectories used for training
num_train = 4
# sampling frequency
f = 1000
# integration time
T = 20.0
# integration step
dt = 0.01
# Desired number of samples per trajectory
num_samples = 1000
# dynamics order
methods = ['first', 'second', 'ef']
# number of repetitions
reps = 3
# number of neurons per layer
num_neurons = [32, 64] # [8, 16, 32, 64, 128, 256]
# number of layers
num_layers = [1, 2, 3] # [1, 2, 3, 4, 5, 6]
# maximum number of iterations
max_iter = 1e5
# loss tolerance
loss_tol = 1e-10
# cuda
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [3]:
sampled_trajectories, delta_t = [], []
for i in range(num_traj):
    # Load trajectory data
    data = np.loadtxt("../learn_embedding/data/synthetic/" + demo_name + "_" + str(i+1) + ".csv")
    t = np.arange(0, data.shape[0]/f, 1/f)
    # remove last point
    data = data[:-1,:]
    t = t[:-1]
    # subsample
    if num_samples is not None:
        idx = DataProcess.subsample(data[:,:2], num_samples)
        data = data[idx]
        t = t[idx]
    # center
    data[:,:2] -= data[-1,:2]
    # zero velocity last sample
    data[-1,2:4] = np.zeros(2)
    # zero acceleration last sample
    data[-1,-2:] = np.zeros(2)
    # add delta time
    delta_t.append(np.append(t[1:]-t[:-1],0))
    # add trajectory
    sampled_trajectories.append(data)

In [4]:
def train_model(model, train_x, train_y):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-6)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=200, 
                                                           threshold=1e-3, threshold_mode='rel', cooldown=0, 
                                                           min_lr=0, eps=1e-8, verbose=False)
    loss_fun = torch.nn.MSELoss()
    epochs = 0
    prev_loss = 1e6
    
    t0 = time.time()
    while epochs <= max_iter:
        optimizer.zero_grad()
        pred = model(train_x)
        loss = loss_fun(pred,train_y)
        loss.backward()
        optimizer.step()
        scheduler.step(loss)
        if np.abs(loss.item() - prev_loss) <= loss_tol:
            break
        prev_loss = loss.item()
        epochs += 1
    return loss.item(), epochs, time.time()-t0

In [5]:
# cosine similarity kernel
cosine_similarity = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

# loss, epochs, time, rmse, cos, dtw
results_log = np.zeros((6, 3, reps))

for i, method in enumerate(methods):
    for j in range(reps):
        if method=='first':
            model = FirstGeometry(Embedding(FeedForward(dim, [64]*2, 1)), torch.tensor([0.0,0.0]).to(device), SPD(dim)).to(device)
        elif method=='second':
            model = SecondGeometry(Embedding(FeedForward(dim, [64]*2, 1)), torch.tensor([0.0,0.0]).to(device), SPD(dim), SPD(dim)).to(device)
        elif method=='ef':
            model = Dynamics(dim=dim, num_features=200, num_diff=10, attractor=torch.tensor([0.0,0.0]).to(device), length=0.5).to(device)

        
        rand_idx = list(np.random.permutation(num_traj))
        if method == 'second':
            train_x = torch.from_numpy(np.concatenate(list(sampled_trajectories[q][:,:2*dim] for q in rand_idx[:num_train]), axis=0)).float().to(device).requires_grad_(True)
            train_y = torch.from_numpy(np.concatenate(list(sampled_trajectories[q][:,-2:] for q in rand_idx[:num_train]), axis=0)).float().to(device)
        else:
            train_x = torch.from_numpy(np.concatenate(list(sampled_trajectories[q][:,:dim] for q in rand_idx[:num_train]), axis=0)).float().to(device).requires_grad_(True)
            train_y = torch.from_numpy(np.concatenate(list(sampled_trajectories[q][:,dim:2*dim] for q in rand_idx[:num_train]), axis=0)).float().to(device)
        loss, epochs, train_time = train_model(model, train_x, train_y)
    
        rmse, cos, dtw = 0.0, 0.0, 0.0
        for k in rand_idx[num_train:]:
            test_dt = torch.from_numpy(delta_t[k]).float().unsqueeze(-1).to(device)
            if method=='second':
                test_x = torch.from_numpy(sampled_trajectories[k][:,:2*dim]).float().to(device).requires_grad_(True)
                test_y = torch.from_numpy(sampled_trajectories[k][:,-dim:]).float().to(device)
                x0 = test_x[0,:].unsqueeze(0)
        
                output = (test_x[:-1, dim:] + test_dt[:-1]*model(test_x[:-1])).detach()
                streamline = Integrator.second_order(dynamics=model,x0=x0,T=T, dt=dt)[:,0,:]
                
                rmse += (output-test_x[1:, dim:].detach()).square().sum().mean().sqrt()/(num_traj-num_train)
                cos += (1 - cosine_similarity(output, test_x[1:, dim:].detach())).mean()/(num_traj-num_train)
                dtw += dtw_ndim.distance(streamline[:,:dim].cpu().detach().numpy(), test_x[:,:dim].cpu().detach().numpy())/(num_traj-num_train)
            else:
                test_x = torch.from_numpy(sampled_trajectories[k][:,:dim]).float().to(device).requires_grad_(True)
                test_y = torch.from_numpy(sampled_trajectories[k][:,dim:2*dim]).float().to(device)
                x0 = test_x[0,:].unsqueeze(0)
            
                output = model(test_x).detach()
                streamline = Integrator.first_order(dynamics=model,x0=x0,T=T, dt=dt)[:,0,:]
                
                rmse += (output-test_y).square().sum().mean().sqrt()/(num_traj-num_train)
                cos += (1 - cosine_similarity(output, test_y)).mean()/(num_traj-num_train)
                dtw += dtw_ndim.distance(streamline[:,:dim].cpu().detach().numpy(), test_x.cpu().detach().numpy())/(num_traj-num_train)

        results_log[0,i,j] = np.round(loss, 5)
        results_log[1,i,j] = np.round(epochs, 5)
        results_log[2,i,j] = np.round(train_time, 5)
        results_log[3,i,j] = np.round(rmse.item(), 5)
        results_log[4,i,j] = np.round(cos.item(), 5)
        results_log[5,i,j] = np.round(dtw, 5)
        print("M: ", method, "R: ", j, 
              "Loss: ", results_log[0,i,j], "Epochs: ", results_log[1,i,j], "Time: ", results_log[2,i,j], 
              "RMSE: ", results_log[3,i,j], "COS: ", results_log[4,i,j], "DTW: ", results_log[5,i,j])

/home/bernardo/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


M:  first R:  0 Loss:  3e-05 Epochs:  1326.0 Time:  3.49315 RMSE:  0.30787 COS:  0.0031 DTW:  0.11935
M:  first R:  1 Loss:  4e-05 Epochs:  3179.0 Time:  7.16503 RMSE:  0.27149 COS:  0.00383 DTW:  0.09308
M:  first R:  2 Loss:  3e-05 Epochs:  4784.0 Time:  11.24602 RMSE:  0.32812 COS:  0.00386 DTW:  0.11348
M:  second R:  0 Loss:  2e-05 Epochs:  2743.0 Time:  23.53206 RMSE:  0.0177 COS:  0.01122 DTW:  0.17012
M:  second R:  1 Loss:  1e-05 Epochs:  5650.0 Time:  47.59475 RMSE:  0.01814 COS:  0.01293 DTW:  0.05804
M:  second R:  2 Loss:  2e-05 Epochs:  2789.0 Time:  21.66704 RMSE:  0.017 COS:  0.01125 DTW:  0.10933
M:  ef R:  0 Loss:  9e-05 Epochs:  3737.0 Time:  68.57057 RMSE:  0.5152 COS:  0.03458 DTW:  0.25254
M:  ef R:  1 Loss:  47529132032.0 Epochs:  1356.0 Time:  24.06339 RMSE:  11114973.0 COS:  0.97969 DTW:  127683.92788
M:  ef R:  2 Loss:  0.00021 Epochs:  3478.0 Time:  64.37967 RMSE:  2.27145 COS:  0.01866 DTW:  2.67232


In [9]:
loss_data = [results_log[0,0,:], results_log[0,1,:], results_log[0,2,:]]
epochs_data = [results_log[1,0,:], results_log[1,1,:], results_log[1,2,:]]
time_data = [results_log[2,0,:], results_log[2,1,:], results_log[2,2,:]]
rmse_data = [results_log[3,0,:], results_log[3,1,:], results_log[3,2,:]]
cos_data = [results_log[4,0,:], results_log[4,1,:], results_log[4,2,:]]
dtw_data = [results_log[5,0,:], results_log[5,1,:], results_log[5,2,:]]

In [ ]:
fig, ax = plt.subplots()
boxprops = dict(linewidth=2, facecolor='lightgray')  # No edge for the boxes
medianprops = dict(linewidth=2, color='black', linestyle='-')
whiskerprops = dict(linewidth=2, color='black', linestyle='--')  # Dashed whiskers
capprops = dict(linewidth=2, color='black')
flierprops = dict(markerfacecolor='g', marker='D')  # Green diamonds for outliers
boxplot = ax.boxplot(rmse_data, boxprops=boxprops, medianprops=medianprops,
                     whiskerprops=whiskerprops, capprops=capprops, 
                     flierprops=flierprops, patch_artist=True, widths=0.5)
colors = ['lightskyblue', 'mediumpurple', 'coral']
for patch, color in zip(boxplot['boxes'], colors):
    patch.set_facecolor(color)
ax.set_xticks([1, 2, 3], ['LE-1st', 'LE-2nd', 'EF'], fontsize=32)
# ax.set_yticks([],fontsize=12)
ax.tick_params(axis='y', labelsize=24)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
fig.tight_layout()
fig.savefig('../media/evaluation_rmse.png', format='png', dpi=100, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots()
boxprops = dict(linewidth=2, facecolor='lightgray')  # No edge for the boxes
medianprops = dict(linewidth=2, color='black', linestyle='-')
whiskerprops = dict(linewidth=2, color='black', linestyle='--')  # Dashed whiskers
capprops = dict(linewidth=2, color='black')
flierprops = dict(markerfacecolor='g', marker='D')  # Green diamonds for outliers
boxplot = ax.boxplot(cos_data, boxprops=boxprops, medianprops=medianprops,
                     whiskerprops=whiskerprops, capprops=capprops, 
                     flierprops=flierprops, patch_artist=True, widths=0.5)
colors = ['lightskyblue', 'mediumpurple', 'coral']
for patch, color in zip(boxplot['boxes'], colors):
    patch.set_facecolor(color)
ax.set_xticks([1, 2, 3], ['LE-1st', 'LE-2nd', 'EF'], fontsize=32)
# ax.set_yticks([],fontsize=12)
ax.tick_params(axis='y', labelsize=24)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
fig.tight_layout()
fig.savefig('../media/evaluation_cos.png', format='png', dpi=100, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots()
boxprops = dict(linewidth=2, facecolor='lightgray')  # No edge for the boxes
medianprops = dict(linewidth=2, color='black', linestyle='-')
whiskerprops = dict(linewidth=2, color='black', linestyle='--')  # Dashed whiskers
capprops = dict(linewidth=2, color='black')
flierprops = dict(markerfacecolor='g', marker='D')  # Green diamonds for outliers
boxplot = ax.boxplot(dtw_data, boxprops=boxprops, medianprops=medianprops,
                     whiskerprops=whiskerprops, capprops=capprops, 
                     flierprops=flierprops, patch_artist=True, widths=0.5)
colors = ['lightskyblue', 'mediumpurple', 'coral']
for patch, color in zip(boxplot['boxes'], colors):
    patch.set_facecolor(color)
ax.set_xticks([1, 2, 3], ['LE-1st', 'LE-2nd', 'EF'], fontsize=32)
# ax.set_yticks([],fontsize=12)
ax.tick_params(axis='y', labelsize=24)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
fig.tight_layout()
fig.savefig('../media/evaluation_dtw.png', format='png', dpi=100, bbox_inches="tight")

In [7]:
np.save('../data/evaluation_learning', results_log)